In [2]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates



import ind.Alpha191 as a191
from scipy import stats
import statsmodels.api as sm

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
# from sklearnex import patch_sklearn, unpatch_sklearn
# unpatch_sklearn() ##注意，少量数据的线性回归没有优势。慎用，存在内存泄露

from sklearn import linear_model


%load_ext autoreload
%autoreload 2
%aimport ind.Alpha191

def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data.rename(columns = {"open":"Open",'market_value':'cap','industry':'ind'}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    
    df_ids = data.index.get_level_values(0)
    
    benchmark = smpl.get_benchmark('沪深300', start=df_ids.min(), end=df_ids.max()).data
    benchmark = benchmark[['open','close']].reset_index('code',drop=True)
    data['bm_index_open'] = df_ids.map(benchmark['open'])
    data['bm_index_close'] = df_ids.map(benchmark['close'])
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)

def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',codes))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)


In [3]:
# #读取已经缓存的复权数据，并预处理
# l = smpl.get_codes_from_blockname('沪深300', sse='all')
# df_all =  assemble_stocks_by_codes(l)
# df_treated = pretreate_data(df_all)

tail = False
if tail:
    df_all = load_cache('all_tail_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
else:
    df_all = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(df_all)
df_treated = pretreate_data(df_all)

In [5]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
# ids = [92,157,149,191]
# ['000001','601728','000002','601825', '601868']

ids = np.arange(1,192)
np.random.shuffle(ids)

def generate_alpha_factors(fun_ids, df=None, type_tail=False):
    import ind.Alpha191 as a191
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np

    for i in fun_ids:
    # for i in ids:
        fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
        if fun_name != 'alpha030':
            indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
        else:
            indx = getattr(a191, fun_name)(df)
        # indx = excute_for_multidates(df_treated.loc[(slice(None),['000001','601728','000002','601825', '601868']),:], lambda x: getattr(a191, fun_name)(x) ,level=1)
        indx.name = '191_'+fun_name
        if type_tail:
            save_cache('{}_tail'.format(indx.name), indx, cache_type=CACHE_TYPE.FACTOR)
        else:
            save_cache('{}_train'.format(indx.name), indx, cache_type=CACHE_TYPE.FACTOR)
        # print(indx)

worker=6
task = task_chunk_split(ids, worker)
results = parallal_task(worker, generate_alpha_factors, task, df=df_treated, type_tail=tail)


Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:11568  pid:3432


  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
def alpha181_(data, dependencies=['close', 'bm_index_close'], max_window=40):
    # SUM(RET-MEAN(RET,20)-(BANCHMARK_INDEX_CLOSE-MEAN(BANCHMARK_INDEX_CLOSE,20))^2,20)/SUM((BANCHMARK_INDEX_CLOSE-MEAN(BANCHMARK_INDEX_CLOSE,20))^3)
    # 优化：数值取对数，否则ret 跟 index 可能不在一个量级上，导致全部结果趋同。
    bm = np.log(data['bm_index_close'])
    bm_mean = bm - bm.rolling(window=20, min_periods=20).mean()
    # print(bm_mean)
    # bm_mean = pd.DataFrame(data=np.repeat(bm_mean.values.reshape(len(bm_mean.values),1), len(data['close'].columns), axis=1), index=data['close'].index, columns=data['close'].columns)
    ret = np.log(data['close']).pct_change(periods=1)
    part1 = (ret-ret.rolling(window=20,min_periods=20).mean()-bm_mean**2).rolling(window=20,min_periods=20).sum()
    part2 = (bm_mean ** 3).rolling(window=20,min_periods=20).sum()

    return part1 / part2

In [62]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(excute_for_multidates(df_treated.loc[(slice(None),['000001','601728','000002','601825', '601868']),], lambda x: alpha181_(x) ,level=1).sort_index())

# excute_for_multidates(df_treated.loc[(slice(None),['000001','000017', '000009']),], lambda x: alpha181_(x) ,level=1).sort_index()

import ind.Alpha191 as a191
# excute_for_multidates(df_treated.loc[(slice(None),['000001','000017', '000009']),], lambda x: alpha181_(x) ,level=1).sort_index()
excute_for_multidates(df_treated.loc[(slice(None),['000001','000017', '000009']),], lambda x: a191.alpha181(x) ,level=1).sort_index()




            code  
2016-01-04  000001            NaN
            000009            NaN
            000017            NaN
2016-01-05  000001            NaN
            000009            NaN
                         ...     
2020-12-30  000009    1498.130431
            000017   -1322.056989
2020-12-31  000001     -89.819438
            000009     473.239417
            000017    -578.708638
Length: 3621, dtype: float64

In [61]:
# xx2 = load_cache('191_alpha181_train',cache_type=CACHE_TYPE.FACTOR).sort_index()
xx2.loc[(slice(None),['000001','000017', '000009']),].sort_index()

191_alpha181
date       code                
2016-01-04 000001           NaN
           000009           NaN
           000017           NaN
2016-01-05 000001           NaN
           000009           NaN
...                         ...
2020-12-30 000009   1498.130431
           000017  -1322.056989
2020-12-31 000001    -89.819438
           000009    473.239417
           000017   -578.708638

[3621 rows x 1 columns]